In [1]:
import sys
import os 
import torch

from torch import nn
from torch.utils.data import DataLoader
import numpy as np
sys.path.append("/home/local/kyeonghunjeong_920205/nipa_bu/COVID19/3.analysis/9.MIL/scAMIL_cell/scMILD")
from utils import *
from dataset import *
from model import *
import pandas as pd
from sklearn.model_selection import train_test_split
import scanpy as sc
from scipy import sparse
import modin.pandas as pd
import ray
ray.init()

2024-05-16 08:34:07,799	WARNING utils.py:575 -- Detecting docker specified CPUs. In previous versions of Ray, CPU detection in containers was incorrect. Please ensure that Ray has enough CPUs allocated. As a temporary workaround to revert to the prior behavior, set `RAY_USE_MULTIPROCESSING_CPU_COUNT=1` as an env var before starting Ray. Set the env var: `RAY_DISABLE_DOCKER_CPU_WARNING=1` to mute this warning.
2024-05-16 08:34:08,343	WARNING services.py:1996 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67104768 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=10.24gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2024-05-16 08:34:08,520	INFO worker.py:1715 -- Started a local Ray instance. View the dashboard 

Python version:,3.8.18
Ray version:,2.9.0
Dashboard:,http://127.0.0.1:8265


In [2]:
os.getcwd()

'/home/local/kyeonghunjeong_920205/nipa_bu/COVID19/3.analysis/9.MIL/scAMIL_cell/scMILD/downstream/NS'

In [3]:
dir_path="NS"
base_path = f"../../data/{dir_path}/"
target_dir = f'{base_path}/AE/'


In [4]:
device_num = 4
device = torch.device(f'cuda:{device_num}' if torch.cuda.is_available() else 'cpu')
print("INFO: Using device: {}".format(device))


INFO: Using device: cuda:4


In [5]:

dat = sparse.load_npz(os.path.join(base_path, "RawCounts.npz"))
genes = open(os.path.join(base_path, "genes.txt")).read().strip().split("\n")
barcodes = open(os.path.join(base_path, "barcodes.txt")).read().strip().split("\n")
meta = pd.read_csv(os.path.join(base_path, "20210701_NasalSwab_MetaData.txt"), sep="\t").drop(axis=0,index=0).reset_index(drop=True)

cell_types = pd.read_csv(os.path.join(base_path, "20210220_NasalSwab_UMAP.txt"), sep="\t").drop(axis=0,index=0).reset_index(drop=True)["Category"]
ct_id = sorted(set(cell_types))
mapping_ct = {c:idx for idx, c in enumerate(ct_id)}

X = []
y = []
ct = []

adata = sc.AnnData(dat.astype(np.float32), obs=barcodes, var=genes)

print(adata.shape)
barcodes = adata.obs[0].tolist()

meta_subset = meta[meta['NAME'].isin(barcodes)]
meta_subset.set_index('NAME', inplace=True)
meta_subset = meta_subset.reindex(adata.obs[0])

adata.obs['ind_cov'] = meta_subset['donor_id'].values
adata.obs['ct_cov'] = meta_subset['Coarse_Cell_Annotations'].values
adata.obs['disease_cov'] = meta_subset['disease__ontology_label'].values

adata = adata[adata.obs['disease_cov'].isin(['normal', 'COVID-19'])]
print(adata.shape)

sc.pp.filter_genes(adata, min_cells=5)
print("Preprocessing Complete!")
print(adata.shape)
mapping = {'normal': 0, 'COVID-19': 1}
adata.obs['disease_numeric'] = adata.obs['disease_cov'].map(mapping)
adata.obs['sample_id_numeric'], _ = pd.factorize(adata.obs['ind_cov'])
sample_labels = adata.obs[['disease_numeric', 'sample_id_numeric']].drop_duplicates()


(32588, 32871)
(26947, 32871)


Preprocessing Complete!
(26947, 27765)


In [6]:

# saved_model_path = '../../results/model_NS_ae_ed128_md64_lr0.0001_500_0.3_3_45_reported'
saved_model_path = '../../results/model_NS_ae_ed128_md64_lr0.0001_500_0.3_500_45_baseline'


In [7]:
for exp in range(1,9):
    print(f'Experiment {exp}')
    train_dataset, val_dataset, test_dataset, label_encoder = load_dataset_and_preprocessors(base_path, exp, device)
    instance_test_dataset = update_instance_labels_with_bag_labels(test_dataset, device)
    model_teacher = torch.load(f'{saved_model_path}/model_teacher_exp{exp}.pt')
    model_encoder = torch.load(f'{saved_model_path}/model_encoder_exp{exp}.pt')
    
    
    model_encoder.to(device)
    model_teacher.to(device)

    model_encoder.eval()
    model_teacher.eval()
    
    with torch.no_grad():
        features = model_encoder(instance_test_dataset.data.clone().detach().float().to(device))[:, :model_teacher.input_dims].detach().requires_grad_(False)
        cell_score_teacher = model_teacher.attention_module(features).squeeze(0)
    features_np = features.cpu().detach().numpy()
    cell_score_teacher_np = cell_score_teacher.cpu().detach().numpy()

    df = pd.DataFrame(features_np, columns = [f'feature_{i}' for i in range(features_np.shape[1])])

    df['cell_type']= label_encoder.inverse_transform(instance_test_dataset.instance_labels.cpu().detach().numpy())
    df['cell_score'] = cell_score_teacher_np
    df['bag_labels'] = instance_test_dataset.bag_labels.cpu().detach().numpy()
    df['instance_labels'] = instance_test_dataset.instance_labels.cpu().detach().numpy()
    df['cell_score_minmax']= (df['cell_score'].values - min(df['cell_score'].values)) / (max(df['cell_score'].values)- min(df['cell_score'].values))

    df.to_csv(f'cell_score_{exp}_baseline.csv', index=False)
    



Experiment 1


Experiment 2


Experiment 3


Experiment 4


Experiment 5


Experiment 6


Experiment 7


Experiment 8


*** SIGTERM received at time=1715851382 on cpu 173 ***
PC: @     0x7f465346368e  (unknown)  epoll_wait
    @     0x7f46536a4420  (unknown)  (unknown)
[2024-05-16 09:23:02,677 E 2577847 2577847] logging.cc:361: *** SIGTERM received at time=1715851382 on cpu 173 ***
[2024-05-16 09:23:02,677 E 2577847 2577847] logging.cc:361: PC: @     0x7f465346368e  (unknown)  epoll_wait
[2024-05-16 09:23:02,678 E 2577847 2577847] logging.cc:361:     @     0x7f46536a4420  (unknown)  (unknown)


: 

In [10]:
torch.cuda.empty_cache()
ray.shutdown()